# Agents as Tools with Strands Agents SDK and Claude 4 Interleaved Thinking

This notebook demonstrates how to use Strands Agents SDK with Claude 4's **interleaved thinking** capability to orchestrate intelligent workflows with specialist agents.

## Understanding Interleaved Thinking

### What is Interleaved Thinking?

Interleaved thinking is a new capability in Claude 4 models that allows the model to:

1. **Think between tool calls**: Process and reason about results before deciding next steps
2. **Chain multiple tools with reasoning**: Make sophisticated multi-step decisions
3. **Adapt strategies dynamically**: Change approach based on intermediate results

### How It Works

There are a lot of similarities between Agent's event loop implemented with and without interleaved thinking:
```
Query → LLM is thinking -> LLM decides to call a Tool -> Event Loop calls the Tool -> Ouput is sent back to LLM -> [ this continues until LLM no longer needs to call any tools - it rendered the Final Answer ]
```

The main difference you'll notice with the interleaved thinking is that Event loop is acting on LLM's "thoughts", rather than "decisions". Notice the second link in the loop above, called "thinking". In a traditional event loop, the thoughts are hidden. We have to wait until LLM renders either a decision to call a tool or produces the Final Answer. 

In case of interleaved thinking, LLM is "leaking" its thoughts into the even loop while it's still in that second step - "LLM is thinking" - and event loop is configured to executed the tools as soon as LLM "thinks" about doing it. What this means is that by the time LLM is done thinking, it actually has the Final Answer, on the very first "decision". 


### Enabling Interleaved Thinking

To enable this feature with Strands and Bedrock:
- Set `temperature=1` (required when thinking is enabled)
- Add beta header: `"anthropic_beta": ["interleaved-thinking-2025-05-14"]`
- Configure reasoning budget: `"reasoning_config": {"type": "enabled", "budget_tokens": 3000}`

## Setup and Imports

In [1]:
import os
from strands import Agent, tool
from strands.models import BedrockModel
from strands.models import bedrock

bedrock.DEFAULT_BEDROCK_MODEL_ID = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"

## Define Specialist Agents as Tools

First, we are going to create four specialist agents using the Strands `@tool` decorator:
- **Researcher**: Gathers factual information
- **Data Analyst**: Processes and analyzes information
- **Fact Checker**: Verifies information accuracy
- **Report Writer**: Creates polished final documents

In [2]:
# Specialist agents implemented as tools using Strands @tool decorator
@tool
def researcher(query: str) -> str:
    """
    Research specialist that gathers factual information.
    
    Args:
        query: Research question or topic to investigate
        
    Returns:
        Research findings and sources
    """
    # Create a focused research agent
    # Note: Each call creates a fresh agent instance (stateless)
    research_agent = Agent(
        model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",  # Optional: Specify the model ID
        system_prompt="You are a research specialist. Gather factual information and cite sources when possible. Keep responses under 200 words.",
        callback_handler=None  # No streaming for tool agents
    )
    
    # Execute the research task
    result = research_agent(f"Research: {query}")
    return str(result)

In [3]:
@tool
def data_analyst(data: str) -> str:
    """
    Data analyst that processes and analyzes information.
    
    Args:
        data: Raw data or research findings to analyze
        
    Returns:
        Analysis with insights and patterns
    """
    # Analyst agent focuses on extracting insights
    analysis_agent = Agent(
        model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
        system_prompt="You are a data analyst. Extract key insights, identify patterns, and provide analytical conclusions. Focus on actionable insights.",
        callback_handler=None
    )
    
    # Analyze the provided data
    result = analysis_agent(f"Analyze this data and provide insights: {data}")
    return str(result)

In [4]:
@tool
def fact_checker(information: str) -> str:
    """
    Fact checker that verifies information accuracy.
    
    Args:
        information: Claims or data to verify
        
    Returns:
        Fact-check results with accuracy assessment
    """
    # Fact-checking agent for verification
    fact_check_agent = Agent(
        model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
        system_prompt="You are a fact checker. Verify claims, assess credibility, and provide confidence levels. Identify any questionable statements.",
        callback_handler=None
    )
    
    # Verify the information
    result = fact_check_agent(f"Fact-check this information: {information}")
    return str(result)

In [5]:
@tool
def report_writer(analysis: str) -> str:
    """
    Report writer that creates polished final documents.
    
    Args:
        analysis: Analyzed data and insights
        
    Returns:
        Formatted final report
    """
    # Writer agent for professional output
    writer_agent = Agent(
        system_prompt="You are a professional report writer. Create clear, well-structured reports with executive summaries and actionable recommendations.",
        callback_handler=None
    )
    
    # Create the report
    result = writer_agent(f"Create a professional report based on: {analysis}")
    return str(result)

## Claude 4 Orchestrator with Interleaved Thinking

Now we create the orchestrator - a Claude 4 agent that uses interleaved thinking to intelligently coordinate the specialist agents.

### How the Orchestrator Works:

1. Receives a high-level task from the user
2. **Thinks** about what information is needed
3. Calls the researcher tool to gather initial data
4. **Thinks** about the research results and what analysis is needed
5. Calls the data analyst to process findings
6. **Thinks** about accuracy and verification needs
7. May call the fact checker if needed
8. **Thinks** about how to present the findings
9. Calls the report writer for final output
10. **Reflects** on the complete workflow before responding

In [6]:
# Claude 4 Orchestrator with Interleaved Thinking using Strands
class StrandsInterlevedWorkflowOrchestrator:
    def __init__(self):
        # Define the orchestrator system prompt for intelligent workflow coordination
        self.system_prompt = """You are an intelligent workflow orchestrator with access to specialist agents.
        
        Your role is to intelligently coordinate a workflow using these specialist agents:
        - researcher: Gathers factual information on any topic
        - data_analyst: Analyzes data and extracts insights
        - fact_checker: Verifies accuracy of information  
        - report_writer: Creates polished final reports

        """
    
    def run_workflow(self, task: str, enable_interleaved_thinking: bool = True) -> str:
        """Execute an intelligent workflow for the given task.
        
        Args:
            task: The task to complete
            enable_interleaved_thinking: Whether to enable interleaved thinking (default: True)
        
        The orchestrator will:
        1. Understand the task requirements
        2. Think about the best approach
        3. Coordinate specialist agents
        4. Reflect on results between steps
        5. Produce a comprehensive output
        """
        thinking_mode = "WITH interleaved thinking" if enable_interleaved_thinking else "WITHOUT interleaved thinking"
        print(f"\nStarting intelligent workflow {thinking_mode} for: {task}")
        print("=" * 70)
        
        # Configure Claude 4 with or without interleaved thinking via Bedrock
        if enable_interleaved_thinking:
            claude4_model = BedrockModel(
                model_id="us.anthropic.claude-sonnet-4-20250514-v1:0",
                max_tokens=4096,
                temperature=1,  # Required to be 1 when thinking is enabled
                additional_request_fields={
                    # Enable interleaved thinking beta feature
                    "anthropic_beta": ["interleaved-thinking-2025-05-14"],
                    # Configure reasoning parameters
                    "reasoning_config": {
                        "type": "enabled",  # Turn on thinking
                        "budget_tokens": 3000  # Thinking token budget
                    }
                }
            )
        else:
            claude4_model = BedrockModel(
                model_id="us.anthropic.claude-sonnet-4-20250514-v1:0",
                max_tokens=4096,
                temperature=1
            )
        
        # Create the orchestrator agent with Claude 4 and specialist tools
        orchestrator = Agent(
            model=claude4_model,
            system_prompt=self.system_prompt,
            tools=[researcher, data_analyst, fact_checker, report_writer]
        )
        
        prompt = f"""Complete this task using intelligent workflow coordination: {task}

        Instructions:
        1. Think carefully about what information you need to accomplish this task
        2. Use the specialist agents strategically - each has unique strengths
        3. After each tool use, reflect on the results and adapt your approach
        4. Coordinate multiple agents as needed for comprehensive results
        5. Ensure accuracy by fact-checking when appropriate
        6. Provide a comprehensive final response that addresses all aspects
        
        Remember: Your thinking between tool calls helps you make better decisions.
        Use it to plan, evaluate results, and adjust your strategy.
        """
        
        try:
            result = orchestrator(prompt)
            return str(result)
        except Exception as e:
            return f"Workflow failed: {e}"

## Run the Demo

Let's see the orchestrator in action! Watch how it thinking and making tool calling while it's thinking.


In [7]:
# Create the orchestrator
print("Strands Agents SDK: Claude 4 Interleaved Thinking Workflow Demo")
print("=" * 70)

try:
    orchestrator = StrandsInterlevedWorkflowOrchestrator()
    print("✅ Orchestrator initialized successfully!")
except Exception as e:
    print(f"❌ Failed to initialize orchestrator: {e}")

Strands Agents SDK: Claude 4 Interleaved Thinking Workflow Demo
✅ Orchestrator initialized successfully!


In [8]:
# Run the workflow with a test case
test_case = "Analyze the impact of remote work on productivity and provide strategic recommendations"

print(f"📋 Task: {test_case}\n")

try:
    result = orchestrator.run_workflow(test_case)
    
    print(f"\n📊 Workflow Result:")
    print("=" * 70)
    print(result)
except Exception as e:
    print(f"❌ Workflow execution failed: {e}")

📋 Task: Analyze the impact of remote work on productivity and provide strategic recommendations


Starting intelligent workflow WITH interleaved thinking for: Analyze the impact of remote work on productivity and provide strategic recommendations
This is a complex analytical task that requires me to analyze the impact of remote work on productivity and provide strategic recommendations. Let me break this down:

1. First, I need to gather comprehensive research on remote work and productivity impacts
2. Then analyze that data to extract insights and patterns
3. Fact-check key findings to ensure accuracy
4. Finally, create a polished report with strategic recommendations

Let me start with the researcher to gather foundational information about remote work's impact on productivity.I'll help you analyze the impact of remote work on productivity and provide strategic recommendations. This requires a systematic approach using multiple specialists. Let me start by gathering comprehensive res

## Try without Interleaved Thinking

Experiment with calling the orchestrator and disabling interleaved thinking. Observe the difference in the output.

In [9]:
# Now let's try the same task WITHOUT interleaved thinking
print("\n" + "="*70)
print("🔄 Now running the same task WITHOUT interleaved thinking")
print("="*70)

try:
    result_without_thinking = orchestrator.run_workflow(test_case, enable_interleaved_thinking=False)
    
    print(f"\n📊 Workflow Result (Without Interleaved Thinking):")
    print("=" * 70)
    print(result_without_thinking)
except Exception as e:
    print(f"❌ Workflow execution failed: {e}")


🔄 Now running the same task WITHOUT interleaved thinking

Starting intelligent workflow WITHOUT interleaved thinking for: Analyze the impact of remote work on productivity and provide strategic recommendations
I'll analyze the impact of remote work on productivity and provide strategic recommendations using a coordinated workflow approach. Let me start by gathering comprehensive research on this topic.
Tool #1: researcher
Now let me gather more specific data on productivity metrics and business outcomes to get a complete picture.
Tool #2: researcher
Excellent! Now I have comprehensive research data. Let me analyze this information to extract key insights and patterns about remote work's impact on productivity.
Tool #3: data_analyst
Let me fact-check the key statistics and claims from my analysis to ensure accuracy before proceeding to the final report.
Tool #4: fact_checker
Excellent! The fact-checking shows most of my key statistics are accurate, with a few clarifications needed. Now